In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import plotly.express as px

from datetime import datetime
from scipy import stats

#Set Display options
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

In [23]:
#Pick Season
season = "2020-21"

#Import Files
all_players = pd.read_csv (r'./data/' + season + '/Merged_data/all_players_merged.csv', parse_dates= ["news_added"])
#all_teams = pd.read_csv (r'./data/' + season + '/Cleaned_data/all_teams.csv')
all_events = pd.read_csv (r'./data/' + season + '/Cleaned_data/all_events.csv', parse_dates= ["deadline_time"])
player_types = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_types.csv')
game_phases = pd.read_csv(r'./data/' + season + '/Cleaned_data/game_phases.csv')

player_season_history = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_season_history.csv')
player_gameweek_history = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_gameweek_history.csv')
#player_gameweek_history = pd.read_csv(r'./data/2019-20/player_past_history/player_gameweek_history_raw.csv')
player_future_fixture = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_future_fixture.csv', parse_dates = ["kickoff_time"])



In [4]:
player_future_fixture.head()

,fixture_id,fixture_code,home_team_id,team_h_score,away_team_id,team_a_score,event,finished,minutes,provisional_start_time,kickoff_time,event_name,is_home,difficulty,player_id,opponent_team_id,team_id
0,44,2128331,12,NaN,1,NaN,5.0,False,0,False,2020-10-17 16:30:00,Gameweek 5,False,5,1,12,1
1,49,2128336,1,NaN,9,NaN,6.0,False,0,False,2020-10-24 14:00:00,Gameweek 6,True,3,1,9,1
2,64,2128351,13,NaN,1,NaN,7.0,False,0,False,2020-11-01 16:30:00,Gameweek 7,False,4,1,13,1
3,69,2128356,1,NaN,2,NaN,8.0,False,0,False,2020-11-07 15:00:00,Gameweek 8,True,2,1,2,1
4,82,2128369,10,NaN,1,NaN,9.0,False,0,False,2020-11-21 15:00:00,Gameweek 9,False,2,1,10,1


In [44]:
def player_future(player_id):
    data = player_future_fixture[player_future_fixture.player_id == player_id]

    player = all_players[all_players.player_id == player_id]
    team = data.team_id.iloc[0]
    goals_scored = player.iloc[0].goals_scored
    player_name = player.iloc[0]['web_name']

    fig = px.line(
        data, 
        x = 'event_name', 
        y = 'difficulty', 
        title = player_name,
        hover_data=['opponent_team_id'])

    fig.show()

In [45]:
player_future(150)

In [46]:
    opponent_strength = player_future_fixture['difficulty']
    opponent_strength
    #opponent_strength = player_analysis.iloc[:,2]

0        5
1        3
2        4
3        2
4        2
        ..
19322    3
19323    5
19324    2
19325    2
19326    2
Name: difficulty, Length: 19327, dtype: int64

In [60]:
def get_SMA(player_id , count):
    data = player_future_fixture[player_future_fixture.player_id == player_id]
    opponent_strength = data['difficulty']
    # List to store moving average results
    SMA2 = list(range(0, len(opponent_strength) - count))

    # Compute moving average
    for ii in range(len(SMA2)):
        # Previous gw index
        index = range(ii, ii + count)
        # Get data for previous gw
        prev_gw2 = opponent_strength.iloc[index]

        # Sum previous gw
        summation2 = np.sum(prev_gw2)

        # Get average
        avg2 = summation2/count
        # Save results to list
        SMA2[ii] = avg2
         
    # Define column label
    label2 = f"Opponent strength SMA"

    
    # Get corresponding gw for moving_avg
    gameweeks2 = opponent_strength.index[:-count]
    gameweek = data.event_name[:-count]
    
    # Convert list into Pandas Series
    SMA2 = pd.Series(SMA2, name = label2, index = gameweek)
    
    
    return SMA2

In [61]:
def player_future_avg(player_id , no_games):
    data = get_SMA(player_id , no_games)

    player = all_players[all_players.player_id == player_id]

    player_name = player.iloc[0]['web_name']
    title = "Average difficulty next " + str(no_games) + " games for " + str(player_name)

    fig = px.line(
        data, 
        title = title)

    fig.show()

In [59]:
player_future_avg(200 , 6)

In [54]:
result = get_SMA(150,4)
result

event_name
Gameweek 9     3.25
Gameweek 10    2.75
Gameweek 11    2.75
Gameweek 12    2.75
Gameweek 13    2.50
Gameweek 14    3.00
Gameweek 15    3.25
Gameweek 16    3.50
Gameweek 17    3.75
Gameweek 18    3.25
Gameweek 19    3.25
Gameweek 20    2.75
Gameweek 21    2.50
Gameweek 22    2.50
Gameweek 23    2.25
Gameweek 24    2.75
Gameweek 25    2.50
Gameweek 26    3.25
Gameweek 27    3.50
Gameweek 28    3.50
Gameweek 29    3.75
Gameweek 30    3.00
Gameweek 31    2.75
Gameweek 32    2.25
Gameweek 33    2.25
Gameweek 34    2.75
Gameweek 35    2.75
Gameweek 36    2.75
Gameweek 37    2.75
Gameweek 38    2.50
Name: Opponent strength SMA, dtype: float64

## FANTASY PLAYER VALUE

In [12]:
all_players.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round',
       'player_code', 'cost_change_event', 'cost_change_event_fall',
       'cost_change_start', 'cost_change_start_fall', 'dreamteam_count',
       'player_type_id', 'ep_next', 'ep_this', 'event_points', 'form',
       'player_id', 'in_dreamteam', 'news', 'news_added', 'now_cost', 'photo',
       'PPG', 'selected_by_percent', 'status', 'team_id', 'team_code',
       'total_points', 'transfers_in_event', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'influence_rank', 'influence_rank_type', 'creativity_rank',
       'creativity_rank_type', 'threat_rank', 'threat_rank_type',
       'ict_index_rank', 'ict_index_rank_type',
       'corners_and

In [13]:
top_players = all_players[['web_name' , 'now_cost', 'status' , 'player_type_id', 'total_points' , 'minutes' , 'PP90' , 'PPG' , 'PPMM' , 'VAPM']][all_players['minutes']>100][all_players['status'] != "i"].sort_values(by='VAPM' , ascending = False).head(60)
top_players.head(20)

,web_name,now_cost,status,player_type_id,total_points,minutes,PP90,PPG,PPMM,VAPM
17,Grealish,70,a,3,35,270,11.666667,11.7,1.671429,1.385714
47,Jorginho,51,a,3,27,252,9.642857,9.0,1.764706,1.372549
509,Martínez,46,a,1,22,270,7.333333,7.3,1.586957,1.152174
252,Konsa,46,a,2,22,270,7.333333,7.3,1.586957,1.152174
249,Mings,50,a,2,23,270,7.666667,7.7,1.540000,1.140000
15,Hourihane,60,a,3,17,137,11.167883,8.5,1.416667,1.083333
279,Zouma,50,a,2,22,270,7.333333,7.3,1.460000,1.060000
189,Son,90,a,3,45,297,13.636364,11.2,1.244444,1.022222
482,Wilson,64,a,4,34,354,8.644068,8.5,1.328125,1.015625
445,Watkins,59,a,4,23,270,7.666667,7.7,1.305085,0.966102


In [14]:
top_players[top_players['player_type_id'] == 1].head(3)

,web_name,now_cost,status,player_type_id,total_points,minutes,PP90,PPG,PPMM,VAPM
509,Martínez,46,a,1,22,270,7.333333,7.3,1.586957,1.152174
545,Darlow,50,a,1,19,360,4.750000,4.8,0.960000,0.560000
551,McCarthy,45,a,1,17,360,4.250000,4.2,0.933333,0.488889


In [15]:
top_players[top_players['player_type_id'] == 2].head(5)

,web_name,now_cost,status,player_type_id,total_points,minutes,PP90,PPG,PPMM,VAPM
252,Konsa,46,a,2,22,270,7.333333,7.3,1.586957,1.152174
249,Mings,50,a,2,23,270,7.666667,7.7,1.540000,1.140000
279,Zouma,50,a,2,22,270,7.333333,7.3,1.460000,1.060000
422,Masuaku,45,a,2,19,269,6.356877,6.3,1.400000,0.955556
423,Balbuena,44,a,2,12,180,6.000000,6.0,1.363636,0.909091


In [16]:
top_players[top_players['player_type_id'] == 3].head(5)

,web_name,now_cost,status,player_type_id,total_points,minutes,PP90,PPG,PPMM,VAPM
17,Grealish,70,a,3,35,270,11.666667,11.7,1.671429,1.385714
47,Jorginho,51,a,3,27,252,9.642857,9.0,1.764706,1.372549
15,Hourihane,60,a,3,17,137,11.167883,8.5,1.416667,1.083333
189,Son,90,a,3,45,297,13.636364,11.2,1.244444,1.022222
18,McGinn,55,a,3,22,270,7.333333,7.3,1.327273,0.963636


In [17]:
top_players[top_players['player_type_id'] == 4].head(3)

,web_name,now_cost,status,player_type_id,total_points,minutes,PP90,PPG,PPMM,VAPM
482,Wilson,64,a,4,34,354,8.644068,8.5,1.328125,1.015625
445,Watkins,59,a,4,23,270,7.666667,7.7,1.305085,0.966102
469,Bamford,58,a,4,30,310,8.709677,7.5,1.293103,0.948276


In [18]:
price_gk = top_players['now_cost'][top_players['player_type_id'] == 1].head(2).sum()
price_def = top_players['now_cost'][top_players['player_type_id'] == 2].head(5).sum()
price_mid = top_players['now_cost'][top_players['player_type_id'] == 3].head(5).sum()
price_fwd = top_players['now_cost'][top_players['player_type_id'] == 4].head(3).sum()


team_cost = price_gk + price_def + price_mid + price_fwd
print("Price for the best team is " + str(np.divide(team_cost,10)))

Price for the best team is 83.8


In [21]:
def player_value():

    fig = px.scatter(
        top_players.head(30), 
        x = 'now_cost', 
        y = 'VAPM',
        #size = 'minutes',
        color = 'total_points',
        text = 'web_name',
        hover_data=['PPG'])

    fig.show()

In [22]:
player_value()